# Grouping and Sorting
This is especially useful when you want to compare groups of data or summarize large amounts of data in a meaningful way.

In [26]:
import math
import pandas as pd

wines_df = pd.read_csv('./wines.csv')
wines_df.columns = wines_df.columns.str.strip()
wines_df.apply(lambda x: x.str.strip() if type(x) is str else x)

def preprocess_country(country):
    country = country.strip()
    return "Unknown" if country == "" else country
wines_df.country = wines_df.country.map(preprocess_country)

wines_df.drop(columns=['Unnamed: 0'], inplace=True)

## `groupby`
The `groupby` method allows you to group rows of data together into a series and call aggregate functions.

Group by _country_, and show the count of wines originating from each country

In [27]:
country_group = wines_df.groupby('country').price.count()
country_group.sort_values(ascending=False)

country
USA        2
Brazil     1
Canada     1
Unknown    1
Name: price, dtype: int64

Group by _country_, and show the lowest price of wines originating from each country

In [28]:
country_group = wines_df.groupby('country').price.min()
country_group.sort_values(ascending=False)

country
Canada      NaN
Unknown      73
USA          37
Brazil       33
Name: price, dtype: object

Group by country and then apply a transformation function. Notice that in order for this to work, we had to preprocess the data and remove the null values from the price column. 

Before:

In [29]:
import logging


def get_best_wine(df):
    return df.loc[df.price.idxmax()]

try:
    country_group = wines_df.groupby(['country']).apply(get_best_wine).sort_values(by='price', ascending=False).drop(columns=['country'])
except:
    logging.error("attempting to group by country")

country_group

ERROR:root:attempting to group by country


country
Brazil       33
Canada      NaN
USA          37
Unknown      73
Name: price, dtype: object

After:

In [30]:
def preprocess_price(price):
    new_price = 0
    if type(price) is str:
         new_price = float(price.replace('$', '').strip())
    elif type(price) is float or type(price) is int:
        new_price = float(price)
    else: 
        raise Exception('Unknown price format')
    if math.isnan(new_price):
        new_price = 0
    return new_price
    
wines_df.price = wines_df.price.map(preprocess_price)

country_group = wines_df.groupby(['country']).apply(get_best_wine).sort_values(by='price', ascending=False).drop(columns=['country'])
country_group

,price,points,comments
country,,,
Unknown,73.0,33,comments 4
USA,55.0,100,comments 1
Brazil,33.0,83,comments 3
Canada,0.0,95,comments 2


#### `agg` a `groupby` method of special note.
This allows you to run multiple aggregate functions at once.

In [32]:
country_group = wines_df.groupby(['country']).price.agg(['min', 'max', 'mean', 'median', 'std', 'count'])
country_group

,min,max,mean,median,std,count
country,,,,,,
Brazil,33.0,33.0,33.0,33.0,NaN,1
Canada,0.0,0.0,0.0,0.0,NaN,1
USA,37.0,55.0,46.0,46.0,12.727922,2
Unknown,73.0,73.0,73.0,73.0,NaN,1


## Multi-indexing
Multi-indexing allows you to group by multiple columns. This is useful when you want to compare groups of data or summarize large amounts of data in a meaningful way. However, this is going to be addressed at a later stage. What's important to know is that if you need to revert back to a single index, you can use the `reset_index()` method.

## Sorting
You can sort by the index or by the values in a column. You can also sort by multiple columns. This is done by invoking the `sort_values()` method.

In [33]:
country_group.sort_values(by='max', ascending=False)

,min,max,mean,median,std,count
country,,,,,,
Unknown,73.0,73.0,73.0,73.0,NaN,1
USA,37.0,55.0,46.0,46.0,12.727922,2
Brazil,33.0,33.0,33.0,33.0,NaN,1
Canada,0.0,0.0,0.0,0.0,NaN,1


You can sort by the index value.

In [34]:
country_group.sort_index()

,min,max,mean,median,std,count
country,,,,,,
Brazil,33.0,33.0,33.0,33.0,NaN,1
Canada,0.0,0.0,0.0,0.0,NaN,1
USA,37.0,55.0,46.0,46.0,12.727922,2
Unknown,73.0,73.0,73.0,73.0,NaN,1


You can sort by more than one column.

In [35]:
country_group.sort_values(by=['max', 'mean'], ascending=False)

,min,max,mean,median,std,count
country,,,,,,
Unknown,73.0,73.0,73.0,73.0,NaN,1
USA,37.0,55.0,46.0,46.0,12.727922,2
Brazil,33.0,33.0,33.0,33.0,NaN,1
Canada,0.0,0.0,0.0,0.0,NaN,1
